<この上にdnn説明>

# CNNを使ったトランプ記号♡♢♤♧分類

### TensorFlow
現在一般的な機械学習ライブラリです(Google開発)

**Installation**
+ [AnacondaでのTensorFlow環境構築と基礎的な使い方](https://miyabi-lab.space/blog/19)

### CNN
簡単に言うと畳み込みをする層を加えたDNNです。(TensorFlowを使えば簡単に実装できます)
+ [定番のConvolutional Neural Networkをゼロから理解する](https://deepage.net/deep_learning/2016/11/07/convolutional_neural_network.html)  
※理解するためには特に複数の書籍や記事を読むことを推奨します。

今回はCNNを用いて画像分類を行います。

### 画像の読み込み
学習用画像データセットとしてkaggleの[Playing Cards](https://www.kaggle.com/datasets/jamesmcguigan/playingcards?resource=download)データセットを使用します。

In [ ]:
from glob import glob
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
dir_path = "./archive"
file_paths = glob(dir_path + "/*.png")
print(cv2.imread(f"{dir_path}/2C.png").shape)
print(cv2.imread(f"{dir_path}/2C.png",cv2.IMREAD_GRAYSCALE).shape)
for file_path in file_paths:
    img = cv2.imread(file_path)
    pltimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #BGR -> RGB
    plt.xticks([]) #軸の非表示 
    plt.yticks([]) #軸の非表示 
    plt.imshow(pltimg)
    plt.show()

### リサイズ + グレースケール処理
手順をわかりやすくするため一度保存します

In [ ]:
export_path = "./archive_gray"
os.mkdir(export_path)
for file_path in file_paths:
    img = cv2.imread(file_path)
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gimg = cv2.resize(gimg,dsize=(200,200))
    file_name = os.path.basename(file_path)
    cv2.imwrite(f"{export_path}/{file_name}", gimg)

### 使用しない画像の除外 + ファイル名をパスから抽出

In [ ]:
export_file_paths = glob(export_path + "/*.png")
export_file_names = [os.path.splitext(os.path.basename(ef_path))[0] for ef_path in export_file_paths if len(os.path.splitext(os.path.basename(ef_path))[0]) <= 3] #学習に使用しないファイルを除外
export_file_names

### 画像-ラベルペアの生成
ここからtensorflow
参考
+ [データ拡張（Data Augmentation）徹底入門！Pythonとkerasでデータ拡張を実装しよう](https://www.codexa.net/data_augmentation_python_keras/)

+ x_lstに画像データ
+ y_lstにラベルデータ(テキスト)

ラベルデータは最終的にベクトルで表します。  
例えば

+ ♡ :(1,0,0,0)
+ ♤ :(0,1,0,0)
+ ♢ :(0,0,1,0)
+ ♧ :(0,0,0,1)

In [ ]:
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense, Activation, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
x_lst = []
y_lst = []
for export_file_name in export_file_names:
    img = cv2.imread(f"{export_path}/{export_file_name}.png",cv2.IMREAD_GRAYSCALE)
    label_key = export_file_name[-1] # C or D or H or S それぞれ記号の頭文字
    x_lst.append(img)
    if label_key == "H":
        y_lst.append(0)
    elif label_key == "S":
        y_lst.append(1)
    elif label_key == "D":
        y_lst.append(2)
    elif label_key == "C":
        y_lst.append(3)

plt_idx = 0
print(y_lst[plt_idx])
pltimg = cv2.cvtColor(x_lst[plt_idx], cv2.COLOR_BGR2RGB) #BGR -> RGB
plt.xticks([]) #軸の非表示 
plt.yticks([]) #軸の非表示 
plt.imshow(pltimg)
plt.show()

##-----

x_train = np.array([x_lst[:-5]]).transpose(1,2,3,0)
x_valid = np.array([x_lst[-5:]]).transpose(1,2,3,0)
y_train = np_utils.to_categorical(y_lst[:-5], 4)
y_valid = np_utils.to_categorical(y_lst[-5:], 4)

x_train.shape

### モデル作成

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,input_shape=(200,200,1)))
model.add(Activation('relu'))
model.add(Conv2D(32,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
adam = Adam(learning_rate=1e-4)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow(x_train, y_train, batch_size = 32, seed = 0)
valid_datagen = ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow(x_valid, y_valid, batch_size = 32, seed=0)

### 学習

In [ ]:
result = model.fit(train_generator,steps_per_epoch=len(x_train) / 32, epochs=30)

### 予測(画像分類)

In [ ]:
idx = 2
print(model.predict(np.array([x_valid[idx]])))
print(y_valid[idx])
pltimg = cv2.cvtColor(x_valid[idx], cv2.COLOR_BGR2RGB) #BGR -> RGB
plt.xticks([]) #軸の非表示 
plt.yticks([]) #軸の非表示 
plt.imshow(pltimg)
plt.show()